In [ ]:
# Make a directory where the train is the dev and the test is the devtest


In [10]:
FLORES_DIR = "/workspace/nllb-flores/floresp-v2.0-rc.3"
OUTPUT_DIR = "../data/flores-200/"

LANGUAGE_PAIRS = [("ban", "en"), ("en", "ban"), ("ban", "id"), ("id", "ban")]
LANGUAGE_PAIRS_NLLB_MAP = {"ban": "ban_Latn", "en": "eng_Latn", "id": "ind_Latn"}



In [16]:
import os
import json

def readlines(filepath):
    with open(filepath) as f:
        return [l for l in f.read().split('\n') if l != ""]

for src, tgt in LANGUAGE_PAIRS:
    os.makedirs(os.path.join(OUTPUT_DIR, f"{src}{tgt}"), exist_ok=True)
    
    src_dev = readlines(os.path.join(FLORES_DIR, "dev", f"dev.{LANGUAGE_PAIRS_NLLB_MAP[src]}"))
    tgt_dev = readlines(os.path.join(FLORES_DIR, "dev", f"dev.{LANGUAGE_PAIRS_NLLB_MAP[tgt]}"))
    src_devtest = readlines(os.path.join(FLORES_DIR, "devtest", f"devtest.{LANGUAGE_PAIRS_NLLB_MAP[src]}"))
    tgt_devtest = readlines(os.path.join(FLORES_DIR, "devtest", f"devtest.{LANGUAGE_PAIRS_NLLB_MAP[tgt]}"))


    # TRAIN: train.src-tgt.json
    with open(os.path.join(OUTPUT_DIR, f"{src}{tgt}", f"train.{src}-{tgt}.json"), 'w') as f:
        translation_dicts = []
        for i in range(len(src_dev)):
            translation_dicts.append({"translation": {
                src: src_dev[i],
                tgt: tgt_dev[i]
            }})

        json.dump(translation_dicts, f)


    with open(os.path.join(OUTPUT_DIR, f"{src}{tgt}", f"valid.{src}-{tgt}.json"), 'w') as f:
        json.dump([{
            "translation": {
                src: src_dev[-1],
                tgt: tgt_dev[-1]
            }
        }], f)
    

    # TEST: test.src-tgt.json + test.src-tgt.src + test.src-tgt.tgt
    with open(os.path.join(OUTPUT_DIR, f"{src}{tgt}", f"test.{src}-{tgt}.{src}"), 'w') as src_f, open(os.path.join(OUTPUT_DIR, f"{src}{tgt}", f"test.{src}-{tgt}.{tgt}"), 'w') as tgt_f:
        for i in range(len(src_devtest)):
            src_f.write(src_devtest[i] + '\n')
            tgt_f.write(tgt_devtest[i] + '\n')
            

    with open(os.path.join(OUTPUT_DIR, f"{src}{tgt}", f"test.{src}-{tgt}.json"), 'w') as f:
        translation_dicts = []
        for i in range(len(src_devtest)):
            translation_dicts.append({"translation": {
                src: src_devtest[i],
                tgt: src_devtest[i]
            }})

        json.dump(translation_dicts, f)
    

    

In [3]:
from datasets import load_dataset

# Replace "bal-Arab" with the name of any other language available in the dataset
# dataset = load_dataset("cis-lmu/glotcc-v1", name="ban-Latn", split="train")
dataset = load_dataset("cis-lmu/Glot500", name="ban_Latn", split="train")


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
list(dataset)[:10]

[{'text': 'Buah delima mungkin salah satu buah yang paling terkenal untuk meningkatkan dorongan seks.',
  'dataset': 'Leipzig_web',
  'script': 'Latn',
  'lang_script': 'Bali,Latn'},
 {'text': 'Ring warsa 2014, akéh krama ring désa Batubulan (proyéksi BPS) kirang langkung 21.443 diri sané kakepah antuk 10.818 lanang miwah 10.625 istri, taler tingkat sex rasionyané kirang langkung 101,82.',
  'dataset': 'Leipzig_wikipedia',
  'script': 'Latn',
  'lang_script': 'Bali,Latn'},
 {'text': 'Dalam pendidikan kritis, guru tidak dianggap sebagai pusat segalanya.',
  'dataset': 'Leipzig_web',
  'script': 'Latn',
  'lang_script': 'Bali,Latn'},
 {'text': 'Kebebasan Kuno taler nénten sami polih ring krama sané akidik miwah homogen, sané dangan antuk kramané mapupul sinarengan ring génah sané pateh anggén mareraosan indik pikobet umum.',
  'dataset': 'nllb_seed',
  'script': 'Latn',
  'lang_script': 'Bali,Latn'},
 {'text': 'Ring Bali, aksara punika nenten tasih pralambang suara /ɻ/, nanging /rə/, sua

In [10]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4")


In [19]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [15]:
import os
os.listdir('../data/ban/opus/ban-id/')

['ban-id.ban.combined.gz',
 'WikiMatrix_v1_xml_id-ban.xml.gz',
 'NLLB_latest_xml_id-ban.xml.gz',
 'NLLB_latest_xml_ban-id.xml.gz',
 'ban-id.ban.preprocessed.gz',
 'ban-id.ban.filtered.gz',
 'WikiMatrix_v1_xml_ban-id.xml.gz',
 'ban-id.ban.valid.gz',
 'ban-id.ban.train.gz',
 'ban-id.ban.test.gz',
 'ban-id.ban.raw.gz',
 'ban-id.ban.preprocessed-dup.gz',
 'ban-id.ban.final.gz',
 'ban-id.ban.temp.gz',
 'ban-id.id.combined.gz',
 'ban-id.id.filtered.gz',
 'ban-id.id.final.gz',
 'ban-id.id.raw.gz',
 'ban-id.id.temp.gz',
 'ban-id.id.preprocessed-dup.gz',
 'ban-id.id.preprocessed.gz',
 'ban-id.id.test.gz',
 'ban-id.nllbsub.id.gz',
 'ban-id.id.train.gz',
 'ban-id.nllbsub.ban.gz',
 'ban-id.wikimatrix.ban.gz',
 'ban-id.id.valid.gz',
 'ban-id.wikimedia.ban.gz',
 'ban-id.wikimatrix.id.gz',
 'ban-id.wikimedia.id.gz',
 'id-ban.ban.final.gz',
 'id-ban.ban.temp.gz',
 'id-ban.ban.valid.gz',
 'id-ban.ban.train.gz',
 'id-ban.id.final.gz',
 'id-ban.id.temp.gz',
 'id-ban.id.test.gz',
 'id-ban.id.train.gz',
 '

In [17]:
import gzip
with gzip.open("../data/ban/opus/ban-id/ban-id.id.preprocessed-dup.gz", "rt") as f:
    st = f.read()

In [22]:
len(st.split('\n')) // 32

2775

In [24]:
(2775*443 + 2746032) * 2

7950714

In [20]:
num_tokens_from_string(st, 'gpt-4')

2746032

In [25]:
$0.075 / 1M input tokens
$0.300 / 1M output tokens

# INPUT:
# 8 MILLION

# OUTPUT:
# 6 MILLION

In [26]:
8* 0.075 + 6 * 0.300

2.4

In [28]:
from datasets import load_dataset
ds = load_dataset("Muennighoff/xP3x", "min_Latn") # Use streaming to not download all at once

The repository for Muennighoff/xP3x contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Muennighoff/xP3x.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]

In [32]:
ds = load_dataset("tatoeba", lang1="min_Latn", lang2="eng_Latn")

The repository for tatoeba contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tatoeba.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


FileNotFoundError: Couldn't find file at https://object.pouta.csc.fi/OPUS-Tatoeba/v2021-07-22/moses/min_Latn-eng_Latn.txt.zip

In [37]:
ds = load_dataset("SEACrowd/minangnlp_mt")

The repository for SEACrowd/minangnlp_mt contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/SEACrowd/minangnlp_mt.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [61]:
sorted(["id", "min"])

['id', 'min']

In [57]:
ds['train'][0]

{'id': '0',
 'src': 'walaupun biaut , kwa katiko diwawancarai straits times pado 2000 , manyabuik bahasonyo anak @-@ anak lee acok bamain di halaman rumah pado malam hari samentaro ayah mereka bamain golf\n',
 'tgt': 'terlepas dari itu , menurut sebuah wawancara dengan straits times 2000 dengan kwa , anak @-@ anak lee sering bermain di halaman rumah pada malam hari sementara ayah mereka bermain golf\n'}

In [40]:
ds["train"][0]

{'id': '0',
 'src': 'walaupun biaut , kwa katiko diwawancarai straits times pado 2000 , manyabuik bahasonyo anak @-@ anak lee acok bamain di halaman rumah pado malam hari samentaro ayah mereka bamain golf\n',
 'tgt': 'terlepas dari itu , menurut sebuah wawancara dengan straits times 2000 dengan kwa , anak @-@ anak lee sering bermain di halaman rumah pada malam hari sementara ayah mereka bermain golf\n'}

In [53]:
import pandas as pd
df = pd.read_csv('https://huggingface.co/datasets/Exqrch/IndonesianNMT/raw/main/id-min.tsv', sep='\t', header=0)

In [70]:
df.dropna()

,Indonesian,Minangkabau
0,Rumah saya di Jakarta sangat besar dan luas de...,Rumah saya di Padang sangat luas dan bagus den...
1,.Saat aku sedang menulis essay di meja belajar...,.Saat nan ado bariso tulis essay di mangaibo a...
2,.Keberhasilan dalam suatu proses pengembangan ...,.Kapayuang lai dalam pagembangan saro lakuak m...
3,"Di pagi hari yang cerah ini, aku bangun dengan...","Dalam sako pagi nan ceria, akhe sakaliyo talam..."
4,"Saya ingin membeli sebuah rumah yang luas, nya...",Saya kamariangan mangaduakan rumah yang pandai...
...,...,...
5056,Banyak orang yang mencari kebahagiaan dalam be...,Banyak urang nan mancari ka bahagiaan dalam ru...
5057,Pertumbuhan ekonomi Indonesia meningkat signif...,Pambagian ekonomi di Minangkabau punyo kambali...
5058,"Sore itu, saat dia sedang melintasi jalan yang...","Sati itam, samentari dia sabalun keluaro baran..."
5059,"Saat cuaca begitu panas, saya suka minum es ke...","Saat udaro begitu pangih, saya sukai minum pin..."


In [47]:
!wget https://huggingface.co/datasets/Exqrch/IndonesianNMT/raw/main/id-min.tsv "id-min.tsv"

--2024-08-08 06:07:14--  https://huggingface.co/datasets/Exqrch/IndonesianNMT/raw/main/id-min.tsv
Resolving huggingface.co (huggingface.co)... 13.225.131.93, 13.225.131.94, 13.225.131.35, ...
Connecting to huggingface.co (huggingface.co)|13.225.131.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2255892 (2.2M) [text/plain]
Saving to: ‘id-min.tsv’

id-min.tsv          100%[===================>]   2.15M  2.44MB/s    in 0.9s    

2024-08-08 06:07:15 (2.44 MB/s) - ‘id-min.tsv’ saved [2255892/2255892]

--2024-08-08 06:07:15--  http://id-min.tsv/
Resolving id-min.tsv (id-min.tsv)... failed: Name or service not known.
wget: unable to resolve host address ‘id-min.tsv’
FINISHED --2024-08-08 06:07:15--
Total wall clock time: 1.2s
Downloaded: 1 files, 2.2M in 0.9s (2.44 MB/s)
